In [59]:
#26/8/24 Creates a K fold KNN model for each file in a folder provided they are padel
import gc
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '999'
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import nnmetrics
from Function_Repo import setbalance
import math
import tensorflow as tf
from tensorflow import keras
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Padel/3D/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
for df in datasets:
    df = df.dropna()
    df = df.drop(['Name'], axis=1)
    df = df.to_numpy()
    df = np.nan_to_num(df)
    df = pd.DataFrame(df)
    df = df.rename(columns={0: "Toxicity_Value"})
    for column in df.columns: 
        df[column] = df[column]  / df[column].abs().max() 
        df[column] = df[column].abs()
    print(df.isnull().values.sum())
    df = df.fillna(0)
    print(df.isnull().values.sum())
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    threshlist = []
    finalresults = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        temp = []
        for item in ytrain:
            temp.append(int(item))
        ytrain = temp
        
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        temp = []
        for item in ytest:
            temp.append(int(item))
        ytest = temp
        
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        temp = []
        for item in yvalid:
            temp.append(int(item))
        yvalid = temp
        
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        
        
        keras.utils.set_random_seed(seed)
        tf.config.experimental.enable_op_determinism()
        length = len(xtrain.columns)
        sampcount = len(ytrain)
        temp = length
        # Encoder Creation
        enc_in = keras.Input(shape=(length, 1))
        x = keras.layers.Flatten()(enc_in)
        # Additional encoder layers
        
        layer1 = math.sqrt(3*sampcount) + 2*(math.sqrt(sampcount/(3)))
        x = keras.layers.Dense(layer1, activation="relu")(x)
        
        layer2 = math.sqrt(sampcount / 3)
        x = keras.layers.Dense(layer2, activation="relu")(x)
        
        enc_out = keras.layers.Dense(1, activation="sigmoid")(x)
        autoencoder = keras.Model(enc_in, enc_out, name='autoencoder')
        # Optimiser
        
        optimise = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

        autoencoder.compile(optimise, loss='binary_crossentropy')
        #autoencoder.summary()
        
        ytrain = np.array(ytrain)
        epochs = 500
        batchsize = 16
        losshistory = []

        minloss = 1
        bestepoch = 0
        for epoch in range(epochs):
            verb = 0
            logs = autoencoder.fit(
            xtrain,
            ytrain,
            verbose = verb,
            epochs=1,
            batch_size=batchsize),
            validation_split=0.2,

            #autoencoder.save(str(f"models/AE-{epoch+1}.keras"))
            loss = logs[0].history['loss'][0]
            losshistory.append(loss)
            if loss < minloss:
                minloss = loss
                bestepoch = epoch

        model_list.append(autoencoder)
    
        preds = autoencoder.predict(xtest)

        epochcount = range(epochs)

        acclist = []
        thresholdlist = []
        maxval = 0
        bestthresh = 0
        for threshold in np.arange(0, 1, 0.01):
            newval = nnmetrics(preds, ytest, threshold)[12]
            acclist.append(newval)
            thresholdlist.append(threshold)
            if newval > maxval:
                maxval = newval
                bestthresh = threshold
        threshlist.append(bestthresh)
        testdat.append(maxval)
        y_values = acclist
        x_values = thresholdlist
        preds = autoencoder.predict(xvalid)
        results = nnmetrics(preds, yvalid, bestthresh)
        validlist.append(results)
        print('for fold', split + 1, 'optimal test mcc of', maxval, 'validation mcc of', results[12])

        iteration = iteration + 1

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        

    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []

    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        fp = np.asarray(fp).astype('float32')
        #appends each model's prediction to a list
        predlist = []
        for model in model_list:
            preds = model.predict(fp, verbose = 0)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        loops = 0
        for modelpreds in predlist:
            if modelpreds < threshlist[loops]:
                zercount = zercount + 1
        loops = loops + 1

        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)

    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    finalresults.append(results[12])
    del autoencoder
    gc.collect
    
    sets += 1

580689
0
For dataset Hepatotoxicity_PaDEL.csv
8/8 [==============================] - 0s 582us/step
for fold 1 optimal test mcc of 0.4929453535882188 validation mcc of 0.30523036902279627
8/8 [==============================] - 0s 586us/step
for fold 2 optimal test mcc of 0.480387043454146 validation mcc of 0.3363193884064238
8/8 [==============================] - 0s 1ms/step
for fold 3 optimal test mcc of 0.5283124595514349 validation mcc of 0.329967414241152
8/8 [==============================] - 0s 895us/step
for fold 4 optimal test mcc of 0.45440912328642313 validation mcc of 0.3851718730578231
8/8 [==============================] - 0s 743us/step
for fold 5 optimal test mcc of 0.37534915958712334 validation mcc of 0.35278847539416985

validation metrics of:
positives in data 293
negatives in data 285
net accuracy = 0.7301038062283737
mcc = 0.46756560897722826
83950
0
For dataset NR-PPAR-gamma_PaDEL.csv
1/1 [==============================] - 0s 9ms/step
for fold 1 optimal test mcc of 

21390
0
For dataset Developmental_Toxicity_PaDEL.csv
1/1 [==============================] - 0s 9ms/step
for fold 1 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 2 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 3 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 4 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 5 optimal test mcc of 0.16666666666666666 validation mcc of 0

validation metrics of:
positives in data 0
negatives in data 0
net accuracy = 0
mcc = 0
154638
0
For dataset NR-ER-LBD_PaDEL.csv
2/2 [==============================] - 0s 985us/step
for fold 1 optimal test mcc of 0.6737856303945685 validation mcc of 0.46668523451860866
2/2 [==============================] - 0s 896us/step
for fold 2 optimal test mcc of 0.5958460099157059 validation mcc of 0.424208015050499

3/3 [==============================] - 0s 648us/step
for fold 3 optimal test mcc of 0.4913530840368859 validation mcc of 0.44428883490488513
3/3 [==============================] - 0s 732us/step
for fold 4 optimal test mcc of 0.3892494720807615 validation mcc of 0.4187081045743367
3/3 [==============================] - 0s 706us/step
for fold 5 optimal test mcc of 0.3875968992248062 validation mcc of 0.17564260704460308

validation metrics of:
positives in data 102
negatives in data 103
net accuracy = 0.6146341463414634
mcc = 0.2339942955759251
345168
0
For dataset NR-ER_PaDEL.csv
5/5 [==============================] - 0s 549us/step
for fold 1 optimal test mcc of 0.44697342086467245 validation mcc of 0.2705606069174096
5/5 [==============================] - 0s 681us/step
for fold 2 optimal test mcc of 0.3431965197520615 validation mcc of 0.3542778756831338
5/5 [==============================] - 0s 502us/step
for fold 3 optimal test mcc of 0.2968218342451796 validation mcc of 0.3134361424

6/6 [==============================] - 0s 464us/step
for fold 3 optimal test mcc of 0.6180195794154728 validation mcc of 0.644886829856412
6/6 [==============================] - 0s 468us/step
for fold 4 optimal test mcc of 0.7128307502061663 validation mcc of 0.5338447358453109
6/6 [==============================] - 0s 558us/step
for fold 5 optimal test mcc of 0.6542443064182195 validation mcc of 0.6379609411111483

validation metrics of:
positives in data 179
negatives in data 243
net accuracy = 0.8104265402843602
mcc = 0.6240078849663715


In [56]:
df.isnull().values.sum()

580689

In [55]:
df

,Toxicity_Value,1,2,3,4,5,6,7,8,9,...,1866,1867,1868,1869,1870,1871,1872,1873,1874,1875
0,0.0,0.0,0.002395,0.000006,0.108939,0.120064,0.000000,0.000000,0.117925,0.112150,...,0.180605,0.585203,0.037485,0.021901,0.051463,0.054569,0.033338,0.029408,0.056509,0.069635
1,0.0,0.0,0.163037,0.026581,0.114625,0.109010,0.000000,0.000000,0.122642,0.112150,...,0.257370,0.326011,0.054239,0.045183,0.030246,0.063495,0.028071,0.023825,0.118486,0.083414
2,1.0,0.0,0.307549,0.094586,0.144822,0.117358,0.000000,0.000000,0.122642,0.242991,...,0.274677,0.189688,0.054458,0.080811,0.078425,0.094688,0.050792,0.043580,0.132458,0.136015
3,0.0,0.0,0.078081,0.006097,0.061235,0.057447,0.000000,0.000000,0.066038,0.056075,...,0.205942,0.508250,0.062499,0.050536,0.039674,0.028904,0.008452,0.008141,0.076966,0.097972
4,1.0,0.0,0.113691,0.012926,0.085193,0.084412,0.000000,0.000000,0.094340,0.140187,...,0.256446,0.332702,0.052127,0.028380,0.043346,0.073572,0.038027,0.031593,0.117739,0.078640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,0.0,0.0,0.205452,0.042211,0.283684,0.360422,0.166667,0.166667,0.353774,0.401869,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2885,0.0,0.0,0.131214,0.017217,0.607382,0.574583,0.000000,0.000000,0.580189,0.551402,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2886,0.0,0.0,0.102579,0.010522,0.141789,0.161789,0.000000,0.000000,0.169811,0.130841,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2887,0.0,0.0,0.169913,0.028871,0.106780,0.103126,0.000000,0.000000,0.113208,0.112150,...,0.165149,0.804574,0.058975,0.054388,0.037548,0.043648,0.021017,0.017781,0.060929,0.096982


In [ ]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Data/Model_Outputs/Fingerprint/NeuralNet_Fingerprint_Results')
resultframe

In [5]:
namelist

['NR-ER_Chemopy.csv',
 'Developmental_Toxicity_Chemopy.csv',
 'NR-aromatase_Chemopy.csv',
 'SR-ATAD5_Chemopy.csv',
 'SR-ARE_Chemopy.csv',
 'SR-p53_Chemopy.csv',
 'Cardiotoxicity-1_Chemopy.csv',
 'SR-HSE_Chemopy.csv',
 'Respiratory_Toxicity_Chemopy.csv',
 'Carcinogenicity_Chemopy.csv',
 'Cardiotoxicity-30_Chemopy.csv',
 'Cardiotoxicity-10_Chemopy.csv',
 'Cardiotoxicity-5_Chemopy.csv',
 'NR-AR-LBD_Chemopy.csv',
 'Ames_Mutagenicity_Chemopy.csv',
 'Hepatotoxicity_Chemopy.csv',
 'NR-AR_Chemopy.csv',
 'Reproductive_Toxicity_Chemopy.csv',
 'NR-ER-LBD_Chemopy.csv',
 'NR-PPAR-gamma_Chemopy.csv',
 'NR-AhR_Chemopy.csv',
 'SR-MMP_Chemopy.csv']

In [23]:
temp = int(ytrain)

TypeError: only size-1 arrays can be converted to Python scalars